<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

In [ ]:
from scipy.stats.sampling import NumericalInverseHermite
from scipy.stats import norm, genexpon
from scipy.special import ndtr
import numpy as np

To create a generator to sample from the standard normal distribution, do:


In [ ]:
class StandardNormal:
    def pdf(self, x):
       return 1/np.sqrt(2*np.pi) * np.exp(-x**2 / 2)
    def cdf(self, x):
       return ndtr(x)

dist = StandardNormal()
urng = np.random.default_rng()
rng = NumericalInverseHermite(dist, random_state=urng)

The `NumericalInverseHermite` has a method that approximates the PPF of the
distribution.


In [ ]:
rng = NumericalInverseHermite(dist)
p = np.linspace(0.01, 0.99, 99) # percentiles from 1% to 99%
np.allclose(rng.ppf(p), norm.ppf(p))

True

Depending on the implementation of the distribution's random sampling
method, the random variates generated may be nearly identical, given
the same random state.


In [ ]:
dist = genexpon(9, 16, 3)
rng = NumericalInverseHermite(dist)
# `seed` ensures identical random streams are used by each `rvs` method
seed = 500072020
rvs1 = dist.rvs(size=100, random_state=np.random.default_rng(seed))
rvs2 = rng.rvs(size=100, random_state=np.random.default_rng(seed))
np.allclose(rvs1, rvs2)

True

To check that the random variates closely follow the given distribution, we can
look at its histogram:


In [ ]:
import matplotlib.pyplot as plt
dist = StandardNormal()
rng = NumericalInverseHermite(dist)
rvs = rng.rvs(10000)
x = np.linspace(rvs.min()-0.1, rvs.max()+0.1, 1000)
fx = norm.pdf(x)
plt.plot(x, fx, 'r-', lw=2, label='true distribution')
plt.hist(rvs, bins=20, density=True, alpha=0.8, label='random variates')
plt.xlabel('x')
plt.ylabel('PDF(x)')
plt.title('Numerical Inverse Hermite Samples')
plt.legend()
plt.show()

Given the derivative of the PDF w.r.t the variate (i.e. ``x``), we can use
quintic Hermite interpolation to approximate the PPF by passing the `order`
parameter:


In [ ]:
class StandardNormal:
    def pdf(self, x):
       return 1/np.sqrt(2*np.pi) * np.exp(-x**2 / 2)
    def dpdf(self, x):
       return -1/np.sqrt(2*np.pi) * x * np.exp(-x**2 / 2)
    def cdf(self, x):
       return ndtr(x)

dist = StandardNormal()
urng = np.random.default_rng()
rng = NumericalInverseHermite(dist, order=5, random_state=urng)

Higher orders result in a fewer number of intervals:


In [ ]:
rng3 = NumericalInverseHermite(dist, order=3)
rng5 = NumericalInverseHermite(dist, order=5)
rng3.intervals, rng5.intervals

(3000, 522)

The u-error can be estimated by calling the `u_error` method. It runs a small
Monte-Carlo simulation to estimate the u-error. By default, 100,000 samples are
used. This can be changed by passing the `sample_size` argument:


In [ ]:
rng1 = NumericalInverseHermite(dist, u_resolution=1e-10)
rng1.u_error(sample_size=1000000)  # uses one million samples

UError(max_error=9.53167544892608e-11, mean_absolute_error=2.2450136432146864e-11)

This returns a namedtuple which contains the maximum u-error and the mean
absolute u-error.

The u-error can be reduced by decreasing the u-resolution (maximum allowed u-error):


In [ ]:
rng2 = NumericalInverseHermite(dist, u_resolution=1e-13)
rng2.u_error(sample_size=1000000)

UError(max_error=9.32027892364129e-14, mean_absolute_error=1.5194172675685075e-14)

Note that this comes at the cost of increased setup time and number of intervals.


In [ ]:
rng1.intervals

1022

In [ ]:
rng2.intervals

5687

In [ ]:
from timeit import timeit
f = lambda: NumericalInverseHermite(dist, u_resolution=1e-10)
timeit(f, number=1)

0.017409582000254886  # may vary

In [ ]:
f = lambda: NumericalInverseHermite(dist, u_resolution=1e-13)
timeit(f, number=1)

0.08671202100003939  # may vary

Since the PPF of the normal distribution is available as a special function, we
can also check the x-error, i.e. the difference between the approximated PPF and
exact PPF
```

```

In [ ]:
import matplotlib.pyplot as plt
u = np.linspace(0.01, 0.99, 1000)
approxppf = rng.ppf(u)
exactppf = norm.ppf(u)
error = np.abs(exactppf - approxppf)
plt.plot(u, error)
plt.xlabel('u')
plt.ylabel('error')
plt.title('Error between exact and approximated PPF (x-error)')
plt.show()